In [11]:
import pandas as pd
# pandas 0.23 ===> raise RuntimeWarning
# pandas 0.17 ====> raise KeyError on pd.merge() function
import numpy as np
from collections import defaultdict
#import matplotlib.pyplot as plt
import os
import re

import scipy.stats as scs
from scipy import spatial

import sklearn
from sklearn.metrics import *

In [12]:
cwd = os.getcwd()
print(cwd)

/home/dce/widedeeptutor/widedeep_diabetes


In [13]:
os.chdir(cwd+"/trainingSet")
diagnosisT = pd.read_csv('training_SyncDiagnosis.csv')
allergyT = pd.read_csv('training_SyncAllergy.csv')
immunT = pd.read_csv('training_SyncImmunization.csv')
labObsT = pd.read_csv('training_SyncLabObservation.csv')
labPanT = pd.read_csv('training_SyncLabPanel.csv')
labResT = pd.read_csv('training_SyncLabResult.csv')
medicationT = pd.read_csv('training_SyncMedication.csv')
patientT = pd.read_csv('training_SyncPatient.csv')
patientCondT = pd.read_csv('training_SyncPatientCondition.csv')
smokingT = pd.read_csv('training_SyncPatientSmokingStatus.csv')
prescripT = pd.read_csv('training_SyncPrescription.csv')
trnscrpT = pd.read_csv('training_SyncTranscript_cleaned.csv')
trnscrpAT = pd.read_csv('training_SyncTranscriptAllergy.csv')
trnscrpDT = pd.read_csv('training_SyncTranscriptDiagnosis.csv')
trnscrptMT = pd.read_csv('training_SyncTranscriptMedication.csv')
myfileT = pd.read_csv('myfile.csv')
conditionT = pd.read_csv('SyncCondition.csv')
smokingT = pd.read_csv('SyncSmokingStatus.csv')

In [14]:
patientT.shape
trnscrpT.shape

(130794, 13)

In [20]:
# 1.1. get features related to BMI (Body Mass Index)
# Bang transcipt da duoc clean cac outliers
# bmi['MeanBMI', 'MaxBMI', 'MinBMI', 'isOverweight', 'isObese', 'BMIDiff']
# 6 features
# Neu khong lay du lieu nam 2012 thi con 9903 patients (45 patients khong co du lieu bmi 3 nam 2009, 2010, 2011)
def makeBMI():
    bmianalysis = trnscrpT.copy()
    bmianalysis['BMI'] = bmianalysis['BMI'].replace(0.000, np.nan) #repalace values 0
    #bmianalysis = bmianalysis[bmianalysis['VisitYear'] != 2012]  #Lay du lieu 3 nam 2009, 2010, 2011
    bmi_mean = bmianalysis.groupby(['PatientGuid'])['BMI'].mean() # get mean BMI of each patient
    bmi_max = bmianalysis.groupby(['PatientGuid'])['BMI'].max() # get max BMI of each patient
    bmi_min = bmianalysis.groupby(['PatientGuid'])['BMI'].min() # get in BMI of each patient
    bmi = pd.concat([bmi_mean, bmi_max, bmi_min], axis = 1)
    bmi.columns = ['MeanBMI', 'MaxBMI', 'MinBMI']
    bmi['isOverweight'] = 0  #Add them cot
    bmi['isObese'] = 0
    bmi.loc[(bmi['MeanBMI'] >= 25) & (bmi['MeanBMI'] < 30), 'isOverweight'] = 1
    bmi.loc[bmi['MeanBMI'] >= 30, 'isObese'] = 1
    bmi['BMIDiff'] = np.abs(bmi['MaxBMI'] - bmi['MinBMI'])
    bmi.fillna(0, inplace=True)
    bmi = bmi.astype(np.float)
    bmi = bmi.reset_index()
    return bmi  
bmi = makeBMI()
#bmi

In [18]:
# 1.2. get features related to Blood Pressure 
# bp['meanSystolicBP', 'meanDiastolicBP', 'maxSystolicBP', 'maxDiastolicBP', 'minSystolicBP', 'minDiastolicBP', 'SystDiff', 'DiastDiff','isPreHyp','isStage1HBP','isStage2HBP']
# 11 features
# Van de: Neu khong lay du lieu blood pressure nam 2012 thi con 9903 patients (45 patients khong co du lieu bp)
def makeBloodPressure():
    bp_mean = trnscrpT.groupby(['PatientGuid'])[['SystolicBP', 'DiastolicBP']].mean()
    bp_max = trnscrpT.groupby(['PatientGuid'])[['SystolicBP', 'DiastolicBP']].max()
    bp_min = trnscrpT.groupby(['PatientGuid'])[['SystolicBP', 'DiastolicBP']].min()
    bp = pd.concat([bp_mean, bp_max, bp_min], axis = 1)
    bp.columns = ['meanSystolicBP', 'meanDiastolicBP', 'maxSystolicBP', 'maxDiastolicBP', 'minSystolicBP', 'minDiastolicBP']
    bp['SystDiff'] = np.abs(bp['maxSystolicBP'] - bp['minSystolicBP'])
    bp['DiastDiff'] = np.abs(bp['maxDiastolicBP'] - bp['minDiastolicBP'])
    bp['isPreHyp'] = 0
    bp['isStage1HBP'] = 0
    bp['isStage2HBP'] = 0
    bp.loc[((bp['meanSystolicBP'] >= 120) & (bp['meanSystolicBP'] < 140)) | ((bp['meanDiastolicBP'] >= 80) & (bp['meanDiastolicBP'] < 90)), 'isPreHyp'] = 1
    bp.loc[((bp['meanSystolicBP'] >= 140) & (bp['meanSystolicBP'] < 160)) | ((bp['meanDiastolicBP'] >= 90) & (bp['meanDiastolicBP'] < 100)), 'isStage1HBP'] = 1
    bp.loc[(bp['meanSystolicBP'] >= 160) | (bp['meanDiastolicBP'] >= 100), 'isStage2HBP'] = 1
    bp.loc[bp['isStage2HBP'] == 1, 'isStage1HBP'] = 0
    bp.loc[bp['isStage1HBP'] == 1, 'isPreHyp'] = 0
    bp.fillna(0, inplace=True)
    bp = bp.astype(np.float)
    bp = bp.reset_index()
    
    return bp  
#bp = makeBloodPressure()
#bp

In [19]:
# Neu khong lay bloodpress nam 2012 con 9903 patients
bpanalysis = trnscrpT.copy()
bpanalysis = bpanalysis[bpanalysis['VisitYear'] < 2012]
bp_mean = bpanalysis.groupby(['PatientGuid'])[['SystolicBP', 'DiastolicBP']].mean()
#bpanalysis
#bp_mean

In [21]:
# 1.3. get features from diagnosis table (cac chan doan benh duoc mo ta boi ma ICD9Code)
# cac features nay lay tu cac ma ICD9code (get dummies)
# Luu y la cac ma ICD9code lien quan den benh tieu duong (gom 250, 250.0, 250.*0 or 250.*2) da bi removed 
# Khong lay du lieu nam 2012
# Nhung patient khong co du lieuj diagnosis se nhan gia tri 0 trong cac cot features ICD9Code tuong ung
# return: 3903 features by ICD9Code dummies (0 no, 1 yes)

def makeDiagnosis():
    diaganalysis = pd.merge(patientT, diagnosisT, on=['PatientGuid'])
    # Khong lay du lieu nam 2012 cua benh nhan
    diaganalysis = diaganalysis[diaganalysis['StartYear'] < 2012]
    diaganalysis = diaganalysis[['PatientGuid', 'DMIndicator', 'ICD9Code']].drop_duplicates()
    diaganalysis.index = diaganalysis['PatientGuid']
    #diaganalysis
    dummies = pd.get_dummies(diaganalysis['ICD9Code'])
    dummies = dummies.groupby(['PatientGuid'])[dummies.columns[1:]].sum() 
    #nhom theo moi PatientGuid, dem tong moi ICD9Code tuong ung cua 1 patient
    dummies = dummies.replace(2,1) #Neu lap 2 lan thay bang 1
    dummies
    patients = patientT.copy()
    patients = patients[['PatientGuid', 'DMIndicator']]

    #diagnosisdata = pd.concat([patients, dummies], axis=1)
    # nhung benh nhan khong co du lieu 3 nam truoc se co cac cot ICD9Code bang 0
    #print(dummies.shape)
    diagnosisdata = pd.merge(patients, dummies, how='outer', on=['PatientGuid']) #pandas 0.17 raises KeyError
    #print(diagnosisdata.shape)
    diagnosisdata['DMIndicator'] = diagnosisdata['DMIndicator'].astype(float)
    diagnosisdata.fillna(0, inplace=True) #import to fill NaN (missing values) to 0
    #total = diagnosisdata.sum(1) # dem so luong diagnosis cua moi patient
    diagnosisdata.iloc[:,2:].astype(np.float)
    return diagnosisdata

data = makeDiagnosis()
data.shape

(9948, 3904)

In [22]:
# 1.4. get features from medications (cac dieu tri benh duoc mo ta boi HL7Text)
# total 2553 features (corresponding to 2553 different medications)
# Bang medication da removed cac medication lien quan den dieu tri tieu duong 
# Khong chi ra du lieu medication theo nam (=> khong can loai bo nam 2012)

def makeMedication():
    medication_analysis = pd.merge(patientT, medicationT, on=['PatientGuid'])
    medication_analysis = medication_analysis[['PatientGuid', 'DMIndicator', 'MedicationName']].drop_duplicates()
    medication_analysis.index = medication_analysis['PatientGuid']
    # here medication_analysis contains 9846 patients because diabetic medications in medication table had been removed 
    # (diabetes medications)
    meddummies = pd.get_dummies(medication_analysis['MedicationName']).reset_index()
    meddummies = meddummies.groupby(['PatientGuid'])[meddummies.columns[1:]].sum()
    # here get dummies medications thanh cac cot (2553 features)
    #medication_analysis
    #print(meddummies.shape)
    patients = patientT[['PatientGuid', 'DMIndicator']]
    patients
    medicationdata = pd.merge(patients, meddummies, how='outer', on=['PatientGuid'])
    # Tron 'outer' se lay 9948 patients, nhung patients khong co medication nao se co cac cot medications bang 0
    # Neu tron 'inner' se chi lay 9846 patients co cac medications
    medicationdata['DMIndicator'] = medicationdata['DMIndicator'].astype(float)
    medicationdata.fillna(0, inplace=True)
    medicationdata.iloc[:, 1:].astype(float)
    #print(medicationdata)
    return medicationdata
medicationdata = makeMedication()
medicationdata.shape
#medicationdata

(9948, 2554)

In [11]:
#### 23/8/2018 Extract cac medication name phan nam trong ngoac (la class/family ???)
medicationana = pd.merge(patientT, medicationT, on=['PatientGuid'])
medicationana = medicationana[['PatientGuid', 'DMIndicator', 'MedicationName']].drop_duplicates()
#medicationana.index = medicationana['PatientGuid']
medicationana
mednames = medicationana['MedicationName'].drop_duplicates()
med_classes = []
for each_medname in mednames:
    start = str(each_medname).find('(')
    end = str(each_medname).find(')')
    if (start >= 0 and end >= 0):
        med_class = str(each_medname)[start + 1:end]
    else:
        med_class = ""
    #print(start, end, med_class)
    if med_class != "" and med_class not in med_classes:
        med_classes.append(med_class)

print(len(mednames))
print(len(med_classes))
med_classes
#file1 = open("medication_all.csv", 'w')
writer = pd.ExcelWriter('medication_all.xlsx')
#mednames = mednames.iloc[:,1]
mednames.to_excel(writer,'Sheet1')
file2 = open("medication_class.csv", 'w')
for item in med_classes:
    file2.write("%s\n" % item)

2553
884


In [39]:
# Chon cac top medication name
meddata = pd.merge(patientT, medicationT, on=['PatientGuid'])
meddata = meddata[['PatientGuid', 'DMIndicator', 'MedicationName']].drop_duplicates() #Con 41677 records
meddata = meddata.dropna()  #Con 41572 records
#meddata['MedicationName'].value_counts()   #Tong so 2552 medication name
meddata['medication_count'] = meddata.groupby('MedicationName')['MedicationName'].transform('count')
medication_freq_thres = 200 
# Co 207 medicationname quan sat duoc hon 50 lan
# Co 85 medicationname quan sat duoc tren 100 lan
# Co 33 medicationname quan sat tren 200 lan
meddata = meddata[meddata['medication_count'] > medication_freq_thres]  
meddata.groupby('MedicationName').count().shape
#meddata


(33, 3)

In [40]:
#Chon cac top diagnosis description
diagdata = pd.merge(patientT, diagnosisT, on=['PatientGuid'])
diagdata = diagdata[diagdata['StartYear'] < 2012]  # 3902 => 3865 diagnosis
diagdata = diagdata[['PatientGuid', 'DiagnosisDescription']].drop_duplicates().dropna() #87674 records

diagdata['diagnosis_count'] = diagdata.groupby('DiagnosisDescription')['DiagnosisDescription'].transform('count')
diagnosis_freq_thres = 100  
#Co 163 diagnosis xuat hien hon 100 lan
#Co 27 diagnosis xuat hien tren 500 lan
diagdata = diagdata[diagdata['diagnosis_count'] > 500]
diagdata.groupby('DiagnosisDescription').count().shape
#diagdata

(27, 2)

In [42]:
#Crossed medication name voi diagnosis description
def makeDiagnosisMedicationCrossed():
    meddiag_data = pd.merge(meddata, diagdata, on=['PatientGuid'])
    meddiag_data = meddiag_data[['PatientGuid', 'DMIndicator', 'DiagnosisDescription', 'MedicationName']].drop_duplicates()
    meddiag_data.index = meddiag_data['PatientGuid']
    crossed_col = ['DiagnosisDescription', 'MedicationName']
    #meddiagdata = meddiagdata.head(6000)
    crossed_colname = '_'.join(crossed_col)
    #print(crossed_colname)
    meddiag_data[crossed_colname] = meddiag_data[crossed_col].apply(lambda x: '-'.join(x), axis=1)
    #meddiag_data[['DiagnosisDescription', 'MedicationName', colname]]
    meddiag_data
    dummies_cols = ['DiagnosisDescription', 'MedicationName', crossed_colname]
    #print(dummies_cols)
    # 27 diagnosis description (>500 observations)+ 85 medicationname (>100 observations) 
    # => 2284 crossed_colname, 6672 patients observations
    # 27 diagnosis description (>500 observations) + 33 medicationname (>200 observations)
    # => 891 crossed_colname, 5571 patients have observations
    #Dummies crossed columns
    meddiag_dummies = pd.get_dummies(meddiag_data[crossed_colname]).reset_index()
    #Neu dummies ca 3 cot diagnosis description, medication name va crossed_colname (=>952 columns)
    #meddiag_dummies = pd.get_dummies(meddiag_data[dummies_cols]).reset_index()
    meddiag_dummies = meddiag_dummies.groupby(['PatientGuid'])[meddiag_dummies.columns[1:]].sum()
    meddiag_dummies
    #print(meddiag_dummies.shape)
    patients = patientT.copy()
    patients = patients[['PatientGuid']]
    meddiagdata = pd.merge(patients, meddiag_dummies, how='outer', on=['PatientGuid'])
    meddiagdata.fillna(0, inplace=True)
    return meddiagdata
meddiagdata = makeDiagnosisMedicationCrossed()
print(meddiagdata.shape)
#meddiagdata

(9948, 892)


In [43]:
# 1.5. get features la cac lab tests (xet nghiem) mo ta boi cot HL7Text
# cac lab tests lien quan den diabetes da duoc removed (insulin, glucose)
# Total co 340 lab tests trong 4 nam, co 791 patients co lab tests (cac patients khac khong co lab tests nao)
# Khong lay du lieu nam 2012 thi con 334 lab tests, con 647 patients co lab tests
# cac patient khong co du lieu lab tests se nhan cac gia tri 0 trong cot lab test tuong ung

def makeLabTest():
    labresult_labpanel = pd.merge(labResT, labPanT, on=['LabResultGuid'])
    labresult_labpanel = labresult_labpanel[['LabResultGuid','PatientGuid', 'LabPanelGuid']].drop_duplicates()
    labresult_panel_obs = pd.merge(labresult_labpanel, labObsT, on=['LabPanelGuid'])
    lab_analysis = labresult_panel_obs[['PatientGuid', 'HL7Text', 'ObservationYear']].drop_duplicates()
    #Khong lay du lieu lab tests trong nam 2012
    lab_analysis = lab_analysis[lab_analysis['ObservationYear'] < 2012]
    lab_analysis.index = lab_analysis['PatientGuid']
    lab_analysis
    labdummies = pd.get_dummies(lab_analysis['HL7Text']).reset_index()
    labdummies = labdummies.groupby(['PatientGuid'])[labdummies.columns[1:]].sum()
    labdummies
    patients = patientT[['PatientGuid', 'DMIndicator']]
    patients
    labdata = pd.merge(patients, labdummies, how='outer', on=['PatientGuid'])
    labdata.fillna(0, inplace=True)
    labdata['DMIndicator'] = labdata['DMIndicator'].astype(float)
    labdata.iloc[:,1:].astype(float)
    return labdata
labdata = makeLabTest()
labdata.shape

(9948, 335)

In [45]:
def heartDisease(text):
    if bool(re.match('41[0-4]|42[0-5]|427|429|74[5-6]', text)) == True:
        return 1
    return 0
def CAD(text):
    if bool(re.match('41[0-4]|429', text)) == True:
        return 1
    return 0
def cardiomyopathy(text):
    if bool(re.match('42[0-5]', text)) == True:
        return 1
    return 0
def CHF(text):
    if bool(re.match('426', text)) == True:
        return 1
    return 0
def arrhythmias(text):
    if bool(re.match('427', text)) == True:
        return 1
    return 0
def heartdefects(text):
    if bool(re.match('74[5-6]', text)) == True:
        return 1
    return 0
def stroke(text):
    if bool(re.match('43[0-1]|43[3-6]|997.02', text)) == True:
        return 1
    return 0
def sleepApnea(text):
    if bool(re.match('727.23|780.57', text)) == True:
        return 1
    return 0
def gestDiab(text):
    if bool(re.match('648.8', text)) == True:
        return 1
    return 0
def polyOvary(text):
    if bool(re.match('256.4', text)) == True:
        return 1
    return 0
def frozenShoulder(text):
    if bool(re.match('726.0', text)) == True:
        return 1
    return 0
def hemochr(text):
    if bool(re.match('275.03', text)) == True:
        return 1
    return 0
def hepatitis(text):
    if bool(re.match('070.2|070.3', text)) == True:
        return 1
    return 0
def diabCompl(text):
    if bool(re.match('250.[1-3]|250.5|250.8|251.[0-2]|270.3|775.6|962.3', text)) == True:
        return 1
    return 0
def kidneyFailure(text):
    if bool(re.match('58[4-5]', text)) == True:
        return 1
    return 0
def dementia(text):
    if bool(re.match('331|290|294|797', text)) == True:
        return 1
    return 0
def acanthosis(text):
    if bool(re.match('701.2', text)) == True:
        return 1
    return 0
def blindness(text):
    if bool(re.match('369', text)) == True:
        return 1
    return 0
def sDysfunction(text):
    if bool(re.match('302.7', text)) == True:
        return 1
    return 0
def preDiabetes(text):
    if bool(re.match('790.29', text)) == True:
        return 1
    return 0

In [46]:
def ICD9Label(text):
    if bool(re.match('14[0-9]|2[0-3][0-9]', text)) == True:
        return 'neoplasms'
    elif bool(re.match('2[4-7][0-9]', text)) == True:
        return 'endoctrine'
    elif bool(re.match('28[0-9]', text)) == True:
        return 'blood'
    elif bool(re.match('29[0-9]|3[0-1][0-9]', text)) == True:
        return 'mental'
    elif bool(re.match('3[2-5][0-9]', text)) == True:
        return 'nervous'
    elif bool(re.match('3[6-8][0-9]', text)) == True:
        return 'sense'
    elif bool(re.match('39[0-9]|4[0-5][0-9]', text)) == True:
        return 'circulatory'
    elif bool(re.match('4[6-9][0-9]|5[0-1][0-9]', text)) == True:
        return 'respiratory'
    elif bool(re.match('5[2-7][0-9]', text)) == True:
        return 'digestive'
    elif bool(re.match('5[8-9][0-9]|6[0-2][0-9]', text)) == True:
        return 'genitourinary'
    elif bool(re.match('6[3-7][0-9]', text)) == True:
        return 'pregnancy'    
    elif bool(re.match('6[8-9][0-9]|70[0-9]', text)) == True:
        return 'skin'   
    elif bool(re.match('7[1-3][0-9]', text)) == True:
        return 'musculoskeletal'       
    elif bool(re.match('7[4-5][0-9]', text)) == True:
        return 'congenital' 
    elif bool(re.match('7[6-7][0-9]', text)) == True:
        return 'perinatal' 
    elif bool(re.match('7[8-9][0-9]', text)) == True:
        return 'symptoms or ill-defined' 
    elif bool(re.match('8[0-9][0-9]|9[0-9][0-9]', text)) == True:
        return 'injuries' 
    elif bool(re.match('E|V', text)) == True:
        return 'suppl' 
    else:
        return 'infectious'

In [47]:
# Function to remove '.' from text (using for removing '.' from ICD9Code values)
def stripPeriods(text):
    result = re.sub('\.', "", text)
    return result

The Practice Fusion Diabetes dataset had several data types removed in order to make the
classification challenge more difficult for the competition

#The first modification was to remove patients that have diagnoses for diabetes complications without a basic diagnosis of type II diabetes. 

#The second modification was to remove ICD9 codes 250, 250.0, 250.*0, 250.*2, 357.2, and 362.0*. The third modification was to remove diabetes medications from the medication records. 

#The final modification was to remove lab tests that identified glucose or insulin related tests.

In [48]:
# 1.6. Get special features from diagnosis table
# Cac features la cac benh dac biet (gia tri 1/0 (yes/no)) can cu theo ma ICD9Code
# Cac chan doan diabetes da duoc removed
# Lay 15 features

def makeSpecials(): 
    data = diagnosisT.copy()
    data['ICD9'] = data['ICD9Code'].apply(stripPeriods)
    data['hasHeartDisease'] = data['ICD9Code'].apply(heartDisease)
    data['hasCAD'] = data['ICD9Code'].apply(CAD)
    data['hasCardiomyo'] = data['ICD9Code'].apply(cardiomyopathy)
    data['hasArrhy'] = data['ICD9Code'].apply(arrhythmias)
    data['hasHeartDefects'] = data['ICD9Code'].apply(heartdefects)
    data['hasCHF'] = data['ICD9Code'].apply(CHF)
    data['hasStroke'] = data['ICD9Code'].apply(stroke)
    data['hasSleepA'] = data['ICD9Code'].apply(sleepApnea)
    data['hasGestDiab'] = data['ICD9Code'].apply(gestDiab)
    data['hasPolyO'] = data['ICD9Code'].apply(polyOvary)
    data['hasFrozenShoulder'] = data['ICD9Code'].apply(frozenShoulder)
    data['hasHemoChr'] = data['ICD9Code'].apply(hemochr)
    data['hasHepatitis'] = data['ICD9Code'].apply(hepatitis)
    #data['hasDiabComp'] = data['ICD9Code'].apply(diabCompl)  #Da removed diabetic diagnosis
    data['hasKidneyF'] = data['ICD9Code'].apply(kidneyFailure)
    data['hasDementia'] = data['ICD9Code'].apply(dementia)
    data['hasAcanthosis'] = data['ICD9Code'].apply(acanthosis)
    data['hasBlindness'] = data['ICD9Code'].apply(blindness)
    data['hasSDysf'] = data['ICD9Code'].apply(sDysfunction)
    data['hasPreDiab'] = data['ICD9Code'].apply(preDiabetes)

    diagnosisSummary = data.groupby(['PatientGuid'])[['hasHeartDisease', 'hasCHF', 'hasStroke', 'hasSleepA', 'hasGestDiab','hasPolyO', 'hasFrozenShoulder', 'hasHemoChr','hasHepatitis', 'hasKidneyF', 'hasDementia','hasAcanthosis', 'hasBlindness' ,'hasSDysf', 'hasPreDiab']].max()
    #groupby theo PatientGuid, max de row nao co (=1) thi lay
    diagnosisSummary = diagnosisSummary.reset_index()
    return diagnosisSummary


In [49]:
# 1.7 get features la cac nhom benh (diabetic type) can cu theo ma ICD9Code tu bang diagnosis
# Co 19 features (see the function: ICD9Label() above)
def makeICD9Labels():
    data = diagnosisT.copy()
    data['ICD9Label'] = data['ICD9Code'].apply(ICD9Label) #get Label of ICD9Code
    data['Counts'] = 1  # Add a column to count the number of ICD9Label of each patient
    ICD9Labeldata = data.groupby(['PatientGuid', 'ICD9Label'])['Counts'].max().reset_index()
    ICD9Labeldata = ICD9Labeldata.pivot('PatientGuid', 'ICD9Label', 'Counts').reset_index()
    ICD9Labeldata.fillna(0, inplace=True) #ICD9Label nao la NaN thi fill thanh 0
    return ICD9Labeldata

#icd9label = makeICD9Labels()
#icd9label

In [50]:
# 1.8. get age of patients by 2012 (year of prediction)
def getPatientAge():
    data = patientT.copy()
    data.drop('PracticeGuid', axis=1, inplace=True)
    data.drop('State', axis=1, inplace=True)
    data['Age'] = 2012 - data['YearOfBirth']
    data.loc[:,'Gender'][data['Gender'] == 'M'] = 1
    data.loc[:,'Gender'][data['Gender'] == 'F'] = 0 
    
    data.drop('YearOfBirth', axis=1, inplace=True)
    data.fillna(0, inplace=True)
    data.iloc[:,1:].astype(float)
    return data

In [69]:
# 2. get data 
# ======================================================= #
patients = getPatientAge()
patients.shape
#patients

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(9948, 4)

In [52]:
bmi_features = makeBMI() # 6 features
bmi_features.shape
#bmi

(9948, 7)

In [53]:
bp_features = makeBloodPressure() # 11 features
bp_features.shape
#bp_features

(9948, 12)

In [55]:
diagnosis_features = makeDiagnosis() # total diagnosis 3903 features
diagnosis_features.shape
#diagnosis_features

(9948, 3904)

In [57]:
medication_features = makeMedication() # total 2553 medication features
#medication_features
medication_features.shape

(9948, 2554)

In [58]:
lab_features = makeLabTest() # total 334 labtests features 
#lab_features
lab_features.shape

(9948, 335)

In [59]:
special_features = makeSpecials() # 15 special features
#special_features
special_features.shape

(9948, 16)

In [60]:
icd9label_features = makeICD9Labels() # 20 icd9label features (diabetic group)
icd9label_features.shape
#icd9label_features

(9948, 20)

In [61]:
diagmedcrossed_features = makeDiagnosisMedicationCrossed() #892 features
diagmedcrossed_features.shape

(9948, 892)

In [62]:
## Use Cosine Similarity to automatically select features 
## cutdown the number of features: 
# diagnosis_features (3903 features)
# medication_features (2553 features)
# lab lab_features (HL7Text) (334 features)
def makeSimilarity():
    columns = diagnosis_features.columns[2:]
    chitest = {}
    cmatrix = {}
    cos = {}
    percent = {}
    for column in columns:
        cm = confusion_matrix(np.array(diagnosis_features['DMIndicator']), np.array(diagnosis_features[column]))
        cmatrix[column] = cm
        chitest[column]= scs.chi2_contingency(cm)[1]
        cos[column] = 1 - spatial.distance.cosine(diagnosis_features['DMIndicator'], diagnosis_features[column])
        percent[column] = [float(cm[1][1])/(float(cm[0][1]) + float(cm[1][1])), (float(cm[0][1]) + float(cm[1][1]))]
    return cos, chitest, cmatrix, percent

cos = makeSimilarity()[0]

In [63]:
# evaluate cosine similarity for selecting diagnosis_features
columns = diagnosis_features.columns[2:]   #col0 - PatientGuid, col1 - DMIndicator
#diagnosis_features.isnull().sum()
cos_dist = {}
for col in columns:
    cos_dist[col] = 1 - spatial.distance.cosine(diagnosis_features['DMIndicator'], diagnosis_features[col])
#cos_dist
#threshold = 0.02 #cutdown from 3903 to 1172
threshold = 0.06
ICD9_list = []
for key in cos_dist.keys():
    if cos_dist[key] > threshold:
        ICD9_list.append(key)
#len(ICD9_list)
new_diagnosis_features = pd.concat([diagnosis_features['PatientGuid'], diagnosis_features[ICD9_list]], axis = 1)
new_diagnosis_features.shape  
#new_diagnosis_features

(9948, 152)

In [65]:
# evaluate cosine similarity for selecting medication_features
#medication_features
#medication_features.isnull().sum()
columns = medication_features.columns[2:] #col0 - PatientGuid, col1 - DMIndicator
cos_dist = {}
for col in columns: 
    cos_dist[col] = 1 - spatial.distance.cosine(medication_features['DMIndicator'], medication_features[col]) 
#cos_dist
#threshold = 0.02 #cutdown from 2553 to 786
threshold = 0.05
medication_list = []
for key in cos_dist.keys():
    if cos_dist[key] > threshold:
        medication_list.append(key)
new_medication_features = pd.concat([medication_features['PatientGuid'], medication_features[medication_list]], axis = 1)
new_medication_features.shape  

(9948, 135)

In [66]:
# evaluate cosine similarity for selecting labtests features
#lab_features.isnull().sum()
lab_features
columns = lab_features.columns[2:] #col0 - PatientGuid, col1 - DMIndicator
cos_dist = {}
for col in columns: 
    cos_dist[col] = 1 - spatial.distance.cosine(lab_features['DMIndicator'], lab_features[col]) 
threshold = 0.02
labtest_list = []
for key in cos_dist.keys():
    if cos_dist[key] > threshold:
        labtest_list.append(key)
new_lab_features = pd.concat([lab_features['PatientGuid'], lab_features[labtest_list]], axis = 1)
new_lab_features.shape
#new_lab_features

(9948, 81)

In [67]:
print(patients.shape, bmi_features.shape, bp_features.shape, special_features.shape, icd9label_features.shape)
print(new_diagnosis_features.shape, new_medication_features.shape, new_lab_features.shape)
print(diagmedcrossed_features.shape)

(9948, 4) (9948, 7) (9948, 12) (9948, 16) (9948, 20)
(9948, 152) (9948, 135) (9948, 81)
(9948, 892)


In [68]:
#=====================================#
#Finally, combine all features

patientdata = pd.merge(patients, bmi_features, how='inner', on =['PatientGuid'])
patientdata = pd.merge(patientdata, bp_features, how='inner', on =['PatientGuid'])
patientdata = pd.merge(patientdata, special_features, how='inner', on =['PatientGuid'])
patientdata = pd.merge(patientdata, icd9label_features, how='inner', on =['PatientGuid'])
patientdata = pd.merge(patientdata, new_diagnosis_features, how='inner', on=['PatientGuid'])
patientdata = pd.merge(patientdata, new_medication_features, how='inner', on=['PatientGuid'])
patientdata = pd.merge(patientdata, new_lab_features, how='inner', on=['PatientGuid'])
patientdata = pd.merge(patientdata, diagmedcrossed_features, how='inner', on=['PatientGuid'])

patientdata.iloc[:,1:].astype(float)
print(patientdata.shape)
patientdata

(9948, 1311)


,PatientGuid,DMIndicator,Gender,Age,MeanBMI,MaxBMI,MinBMI,isOverweight,isObese,BMIDiff,...,"Urinary tract infection, site not specified-Omeprazole oral delayed release capsule","Urinary tract infection, site not specified-PredniSONE oral tablet","Urinary tract infection, site not specified-Septra DS (sulfamethoxazole-trimethoprim) oral tablet","Urinary tract infection, site not specified-Simvastatin oral tablet","Urinary tract infection, site not specified-Singulair (montelukast) oral tablet","Urinary tract infection, site not specified-Synthroid (levothyroxine) oral tablet","Urinary tract infection, site not specified-Vitamin D2 (ergocalciferol) oral capsule","Urinary tract infection, site not specified-Xanax (ALPRAZolam) oral tablet","Urinary tract infection, site not specified-Zithromax Z-Pak (azithromycin) oral tablet","Urinary tract infection, site not specified-Zocor (simvastatin) oral tablet"
0,FB6EFC3D-1A20-4497-9CBD-00027CC5D220,0,1,83,19.047667,19.343,18.900,0.0,0.0,0.443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C6746626-6783-4650-A58F-00065649139A,0,0,27,26.389000,27.025,25.753,1.0,0.0,1.272,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E05C6E8F-779F-4594-A388-000C635AE4D3,0,0,28,38.041000,48.217,31.923,0.0,1.0,16.294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EAEBD216-F847-4355-87B2-000D942E08F0,0,1,53,32.981333,34.241,32.004,0.0,1.0,2.237,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C7F10A80-4934-42D2-8540-000FBEBA75C8,0,0,22,32.917813,34.931,30.432,0.0,1.0,4.499,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3BDB6A99-A404-4E9C-BE4B-002054F8B0F4,0,0,65,39.553429,42.131,37.370,0.0,1.0,4.761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5DA96225-0D61-4F1E-80D8-0025B33669D3,0,1,43,27.220667,27.597,26.952,1.0,0.0,0.645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,D2516A79-B6E2-4830-B637-002B16BA9423,1,0,61,43.165000,43.766,42.564,0.0,1.0,1.202,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,B06DEAB9-9F17-44D2-88D9-0030A48F0077,1,0,88,27.461000,27.461,27.461,1.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,03D53B79-8E30-411D-AE32-003400411BDE,0,0,31,35.141000,35.141,35.141,0.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
#patientdata.iloc[0:1000].to_csv("patientdata_1000.csv")

In [70]:
patientdata.to_csv('patientdata_1311.csv') # auc ~0.82 (medication threshold = 0.05, diagnosis threshold = 0.06)